<a href="https://colab.research.google.com/github/bkristensen/Colabs/blob/main/Teo/Hent_synonymer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installere moduler

In [2]:
!pip install gspread oauth2client pyppeteer nest_asyncio


ERROR: Could not find a version that satisfies the requirement sqlite (from versions: none)
ERROR: No matching distribution found for sqlite


# Importér moduler

In [ ]:
import gspread
import sqlite3

import asyncio
import nest_asyncio

import pyppeteer

import google.auth

# from google.colab import auth

# from pyppeteer import launch
# from pyppeteer import chromium_downloader

[INFO] Starting Chromium download.
INFO:pyppeteer.chromium_downloader:Starting Chromium download.
100%|██████████| 183M/183M [00:00<00:00, 308Mb/s]
[INFO] Beginning extraction
INFO:pyppeteer.chromium_downloader:Beginning extraction
[INFO] Chromium extracted to: /root/.local/share/pyppeteer/local-chromium/1181205
INFO:pyppeteer.chromium_downloader:Chromium extracted to: /root/.local/share/pyppeteer/local-chromium/1181205


TypeError: object NoneType can't be used in 'await' expression

### Test Pypeteer browser

In [ ]:

async def test_browser():
    browser = await pyppeteer.launch(headless=True, args=["--no-sandbox"])
    page = await browser.newPage()
    await page.goto("https://example.com")
    content = await page.content()
    await browser.close()
    print("Browser virker!")

await test_browser()

Browser virker!


# Init

### Opret adgang til Google drev

In [ ]:
# Godkend adgang til Google sheets
google.colab.auth.authenticate_user()

# Brug google.auth i stedet for oauth2client
creds, _ = google.auth.default()
gc = gspread.authorize(creds)

### Hent Pyppeter browser pakke

In [ ]:
nest_asyncio.apply()
await pyppeteer.chromium_downloader.download_chromium()

### Opret SQLite database

In [ ]:
# Opret forbindelse til SQLite-database
conn = sqlite3.connect("synonymer.db")
cursor = conn.cursor()

# Opret tabeller
cursor.executescript("""
CREATE TABLE IF NOT EXISTS Tags (
    TagId INTEGER PRIMARY KEY AUTOINCREMENT,
    Tag STRING(100) UNIQUE NOT NULL
);

CREATE TABLE IF NOT EXISTS Synonyms (
    SynonymId INTEGER PRIMARY KEY AUTOINCREMENT,
    Text STRING(100) UNIQUE NOT NULL
);

CREATE TABLE IF NOT EXISTS TagSynonyms (
    TagId INTEGER NOT NULL,
    SynonymId INTEGER NOT NULL,
    PRIMARY KEY (TagId, SynonymId),
    FOREIGN KEY (TagId) REFERENCES Tags(TagId),
    FOREIGN KEY (SynonymId) REFERENCES Synonyms(SynonymId)
);
""")

conn.commit()


In [ ]:

async def hent_synonymer(ordet):
    url = f"https://synonymet.dk/ord/{ordet}"
    browser = await launch(headless=True, args=["--no-sandbox"])
    page = await browser.newPage()
    await page.goto(url)

    try:
        await page.waitForSelector(".wordcloud-span", timeout=3000)
        elementer = await page.querySelectorAll(".wordcloud-span")
        synonymer = []
        for el in elementer:
            tekst = await page.evaluate('(el) => el.textContent', el)
            if tekst.strip():
                synonymer.append(tekst.strip())
    except:
        synonymer = []

    await browser.close()
    return synonymer

In [ ]:
# Åbn arket og vælg det første ark
sheet = gc.open("Synonymer").sheet1

# Hent alle ord fra kolonne A, start fra række 2
ordliste = sheet.col_values(1)[1:]

In [ ]:
resultater = []
for i, ordet in enumerate(ordliste):
    kol = 2
    print(f"Henter synonymer for: {ordet}")
    syns = asyncio.get_event_loop().run_until_complete(hent_synonymer(ordet))
    if syns:
      for syn in syns:
        kol += 1
        sheet.update_cell(i + 2, kol, syn)

    tekst = ", ".join(syns) if syns else "Ingen synonymer fundet"

    resultater.append(tekst)

    # Skriv i kolonne B (række i + 2 pga. overskrift)
    sheet.update_cell(i + 2, 2, tekst)

Henter synonymer for: Adfærd
Henter synonymer for: Adlyde
Henter synonymer for: Advarsel
Henter synonymer for: Afgud
Henter synonymer for: Afkom
Henter synonymer for: Afstand
Henter synonymer for: Aftale
Henter synonymer for: Agressiv
Henter synonymer for: Alderdom
Henter synonymer for: Alkohol
Henter synonymer for: Anderledes
Henter synonymer for: Anger
Henter synonymer for: Angst
Henter synonymer for: Ansvar
Henter synonymer for: Arbejdspunkt
Henter synonymer for: Artikel
Henter synonymer for: Arve
Henter synonymer for: Bagtaler
Henter synonymer for: Barmhjertig
Henter synonymer for: Begivenhed
Henter synonymer for: Begærlig
Henter synonymer for: Behov
Henter synonymer for: Bekymring
Henter synonymer for: Beskeden
Henter synonymer for: Beskyttelse
Henter synonymer for: Betydning
Henter synonymer for: Bibelen
Henter synonymer for: Bibellæsning
Henter synonymer for: Bidrag
Henter synonymer for: Billedanalyse
Henter synonymer for: Billeddyrkelse
Henter synonymer for: Blasfemi
Henter syn

APIError: APIError: [429]: Quota exceeded for quota metric 'Write requests' and limit 'Write requests per minute' of service 'sheets.googleapis.com' for consumer 'project_number:522309567947'.